In [1]:
import requests
import pandas as pd
import numpy as np
import pyodbc
import psycopg2
from datetime import datetime
from dotenv import load_dotenv
import os
import logging

In [2]:
# Configurar o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [3]:
def clear_env_variables():
    """
    Limpa as variáveis de ambiente específicas usadas para conexão ao banco de dados.
    """
    keys_to_clear = ['SERVER_MIS', 'DATABASE_MIS', 'USERNAME_MIS', 'PASSWORD_MIS', 'DRIVER_MIS', 'PORT_MIS']
    for key in keys_to_clear:
        if key in os.environ:
            del os.environ[key]

In [4]:
def connect_to_database(env_file):
    """
    Carrega variáveis de ambiente de um arquivo .env especificado e estabelece uma conexão com o banco de dados.
    
    Args:
    env_file (str): Caminho para o arquivo .env.
    
    Returns:
    connection: Objeto de conexão com o banco de dados.
    cursor: Objeto cursor para interagir com o banco de dados.
    """

    # Limpar variáveis de ambiente específicas
    clear_env_variables()
    # Carregar variáveis de ambiente do arquivo .env especificado
    load_dotenv(env_file)
    
    # Parâmetros da conexão
    SERVER = os.getenv('SERVER_MIS')
    DATABASE = os.getenv('DATABASE_MIS')
    USERNAME = os.getenv('USERNAME_MIS')
    PASSWORD = os.getenv('PASSWORD_MIS')
    DRIVER = os.getenv('DRIVER_MIS')
    PORT = os.getenv('PORT_MIS') 
    
    # Logar as variáveis carregadas
    logging.info(f"Servidor: {SERVER}")
    logging.info(f"Banco de Dados: {DATABASE}")
    logging.info(f"Usuário: {USERNAME}")


    # Verificar se todas as variáveis de ambiente necessárias foram carregadas
    if not all([SERVER, DATABASE, USERNAME, PASSWORD, DRIVER]):
        raise ValueError("Certifique-se de que todas as variáveis de ambiente (SERVER_MIS, DATABASE_MIS, USERNAME_MIS, PASSWORD_MIS, DRIVER_MIS) estejam definidas no arquivo .env.")
    
    # Conexão com o SQL Server
    connection_string = f'DRIVER={DRIVER};SERVER={SERVER},{PORT};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()
    
    return connection, cursor

"""
Exemplo de uso:

env_file_path = 'path/to/your/.env'  # Substitua pelo caminho para o seu arquivo .env

connection, cursor = connect_to_database(env_file_path)

Agora você pode usar 'connection' e 'cursor' para interagir com o banco de dados
"""


"\nExemplo de uso:\n\nenv_file_path = 'path/to/your/.env'  # Substitua pelo caminho para o seu arquivo .env\n\nconnection, cursor = connect_to_database(env_file_path)\n\nAgora você pode usar 'connection' e 'cursor' para interagir com o banco de dados\n"

In [5]:
# Carrega as variáveis de ambiente do arquivo .env
#load_dotenv(dotenv_path=r"C:\Users\Matheus Alexander\Desktop\DW\Data-Warehouse\Environment\.env")
# Exemplo de uso
if __name__ == "__main__":
    env_file_path = r'C:\Users\int.matheus\Desktop\DW\Data-Warehouse\Environment\.Env_01'  # Substitua pelo caminho para o seu arquivo .env
    connection, cursor = connect_to_database(env_file_path)
    logging.info("Conexão estabelecida com sucesso.")

2024-07-02 14:42:06,388 - INFO - Servidor: 10.10.220.101
2024-07-02 14:42:06,391 - INFO - Banco de Dados: RPL_OLOS
2024-07-02 14:42:06,393 - INFO - Usuário: srvc.mis.prd


2024-07-02 14:42:06,495 - INFO - Conexão estabelecida com sucesso.


In [6]:
data = cursor.execute("SELECT TOP 10 *  FROM Reason")
#data = cursor.execute("SELECT TOP 10 *  FROM CallDetail")

In [7]:
# Obter os dados
columns = [column[0] for column in cursor.description]  # Obter nomes das colunas
data = cursor.fetchall()  # Obter todas as linhas

# Converter os dados para uma lista de dicionários
result = [dict(zip(columns, row)) for row in data]

# Opcional: Converter os dados para um DataFrame do Pandas
df = pd.DataFrame(result)

# Exibir os resultados
display(df)




,Reasonld,Activated,LastModifiedUserld,LastModified,Description
0,100,1,1,2015-11-03 12:38:49.470,Reuniao
1,101,1,1,2015-11-03 12:39:02.187,Almoco
2,102,1,11809,2023-09-05 11:51:25.280,Lanche
3,103,1,1,2015-11-03 12:39:21.797,Saida Particular
4,104,1,1,2015-11-03 12:39:30.893,Banheiro
5,105,1,1,2015-11-03 12:39:51.780,Atendimento Recepcao
6,106,1,1,2015-11-03 12:40:04.940,Feedback
7,107,1,1,2015-11-03 12:40:15.057,Tabulacao
8,108,1,1,2015-11-03 12:40:27.677,Livre
9,109,1,1163,2021-07-20 11:37:30.350,Em Atendimento


In [8]:
# Fechar a conexão
cursor.close()
connection.close()